In [4]:
#mounting to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
import numpy as np
from scipy.signal import butter, filtfilt, welch
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cdist
import joblib


In [5]:
model_path = '/content/drive/MyDrive/EEG_Project/models/pfen_rf_model.pkl'
scaler_path = '/content/drive/MyDrive/EEG_Project/models/scaler.pkl'
clf = joblib.load(model_path)
scaler = joblib.load(scaler_path)

print("✅ Model and scaler loaded successfully.")

✅ Model and scaler loaded successfully.


In [6]:
#preprocessing
def bandpass_filter(sig, low=0.5, high=50, fs=173.61, order=4):
    nyq = 0.5 * fs
    low_cut = low / nyq
    high_cut = high / nyq
    b, a = butter(order, [low_cut, high_cut], btype='band')
    return filtfilt(b, a, sig)

def sliding_windows(sig, window_s=2.0, overlap=0.6, fs=173.61):
    win_samples = int(window_s * fs)
    step = int(win_samples * (1 - overlap))
    windows = []
    for start in range(0, len(sig) - win_samples + 1, step):
        windows.append(sig[start:start + win_samples])
    return windows

def ordinal_patterns_vectorized(seq, m=3, tau=1):
    N = len(seq) - (m - 1) * tau
    if N <= 0:
        return np.empty((0, m), dtype=int)
    patterns = np.empty((N, m), dtype=int)
    for i in range(N):
        vec = seq[i:i + m * tau:tau]
        ranks = np.argsort(np.argsort(vec))
        patterns[i] = ranks
    return patterns

def phi_from_patterns(patterns, r, n):
    N = patterns.shape[0]
    if N <= 1:
        return 0.0
    D = cdist(patterns, patterns, metric='chebyshev')
    iu = np.triu_indices(N, k=1)
    mu = np.exp(- (D[iu] ** n) / r)
    return np.mean(mu) if mu.size > 0 else 0.0

def permutation_fuzzy_entropy(window, m=3, tau=1, n=2, r_factor=0.2):
    std = np.std(window)
    if std == 0:
        return 0.0
    r = max(1e-12, r_factor * std)
    Pm = ordinal_patterns_vectorized(window, m=m, tau=tau)
    Pm1 = ordinal_patterns_vectorized(window, m=m+1, tau=tau)
    phi_m = phi_from_patterns(Pm, r, n)
    phi_m1 = phi_from_patterns(Pm1, r, n)
    if phi_m <= 0 or phi_m1 <= 0:
        return 0.0
    val = -np.log(phi_m1 / phi_m)
    return float(val) if np.isfinite(val) else 0.0

def bandpower_welch(window, fs=173.61, band=(1,4)):
    f, Pxx = welch(window, fs=fs, nperseg=min(len(window), 256))
    idx = np.logical_and(f >= band[0], f <= band[1])
    return np.trapz(Pxx[idx], f[idx]) if np.any(idx) else 0.0

def features_from_window(window, fs=173.61):
    m = np.mean(window)
    s = np.std(window)
    sk = skew(window)
    kt = kurtosis(window)
    pfen_val = permutation_fuzzy_entropy(window, m=3, tau=1, n=2, r_factor=0.2)
    delta = bandpower_welch(window, fs=fs, band=(0.5,4))
    theta = bandpower_welch(window, fs=fs, band=(4,8))
    alpha = bandpower_welch(window, fs=fs, band=(8,13))
    beta  = bandpower_welch(window, fs=fs, band=(13,30))
    total = delta + theta + alpha + beta + 1e-12
    d_rel = delta/total; t_rel = theta/total; a_rel = alpha/total; b_rel = beta/total
    return [pfen_val, m, s, sk, kt, delta, theta, alpha, beta, d_rel, t_rel, a_rel, b_rel]

In [7]:
def extract_features_for_signal(sig, fs=173.61):
    sig_f = bandpass_filter(sig, fs=fs)
    windows = sliding_windows(sig_f, fs=fs)
    window_feats = [features_from_window(w, fs=fs) for w in windows]
    window_feats = np.array(window_feats)
    if window_feats.shape[0] == 0:
        return np.zeros(13 * 5)
    agg = []
    for j in range(window_feats.shape[1]):
        col = window_feats[:, j]
        agg.extend([np.nanmean(col), np.nanstd(col),
                    np.nanpercentile(col,25), np.nanpercentile(col,50), np.nanpercentile(col,75)])
    return np.array(agg)

In [13]:
new_file_path = '/content/drive/MyDrive/EEG_Project/datasets/FNOSZ/F/F067.txt'  # <-- replace with your test file

sig = np.loadtxt(new_file_path)
features = extract_features_for_signal(sig)

X_new_scaled = scaler.transform(features.reshape(1, -1))
pred = clf.predict(X_new_scaled)[0]

label_names = {0: 'F', 1: 'N', 2: 'O', 3: 'S', 4: 'Z'}
print("✅ Prediction for", os.path.basename(new_file_path), ":", label_names[pred])
label_meaning = {
    0: "F - Eyes Open (Healthy, Normal EEG)",
    1: "N - Eyes Closed (Healthy, Normal EEG)",
    2: "O - Epileptic, Non-Seizure EEG",
    3: "S - Seizure Detected ⚡ (Epileptic Activity)",
    4: "Z - Healthy, Non-Seizure EEG"
}

print(f"🧠 Meaning: {label_meaning[pred]}")

/tmp/ipython-input-1720778688.py:54: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(Pxx[idx], f[idx]) if np.any(idx) else 0.0


✅ Prediction for F067.txt : F
🧠 Meaning: F - Eyes Open (Healthy, Normal EEG)
